# Workflow for Low coverage Individuals
### Aligning, variantcalling and merging of all individuals of a Generation

## overview:
![low_coverage_workflow](../../figures/LOW_COV_workflow.png)

## Run Snakemake pipeline to get per-individual VCFs

config-files for for [F2](./config/config_f2.yaml), [F3](./config/config_f3.yaml), [F8](./config/config_f8.yaml) and [F15](./config/config_f15.yaml)
are in the  [config folder](./config) 

note that you have to modify the snakemake file to use the corresponding config-file


## run batch-merge twice:
here for F8  
the Code below is also in the script [make_merge_stage1.sh](./F8/make_merge_stage1.sh)

In [ ]:
%%bash
source activate v3 
mkdir ./sbatches_merge_1
mkdir /home/tilman/storage/F8/merge_vcf_stage1

python create_merge.py --infolder /home/tilman/storage/F8/bams/ \
                       --prefix /home/tilman/storage/F8/merge_vcf_stage1/run001 \
                       --make_sbatch \
                       --threads 4 \
                       --bcftools_path bcftools \
                       --batchsize 10 \
                       --scriptprefix ./sbatches_merge_1/stage1

#execute : 
ls ./sbatches_merge_1/stage1* | xargs -n1 -I{} bash -c "sbatch  {}; sleep 2s" # sbatch had i/o issues

this uses the script [create_merge.py](./create_merge.py) to create an sbatch file for each merge job submit those.
each merge-job merges batchsize=10 files and uses four cores for each of these jobs.

the subsequent ten-sample files are merged in the next stage, using [make_merge_stage2.sh](./F8/make_merge_stage2.sh) 

which also executes [create_merge.py](./create_merge.py), but on the merged files, with a smaller batchsize and more cores (mostly because of memory) per job.  
subsequently, the remainder of this is merged with [merge_final_f8.sh](./F8/merge_final.sh) 

## rename samples

run [sample_name_list.py](./F8/sample_name_list.py) for each generation.  
this file is also used for the sample name generation in stripes.  
run [rename_samples.sh](./F8/rename_samples.sh) for each generation